# Fish Detection and Tracking

In [3]:
import pandas as pd

df = pd.read_csv('detectionsBCIT_fixed.csv')



In [4]:
fish = df.acoustic_tag_id.unique()
fish

array(['A69-1602-64231', 'A69-1602-64232', 'A69-1602-64234',
       'A69-1602-64235', 'A69-1602-64236', 'A69-1602-64237',
       'A69-1602-64238', 'A69-1602-64241', 'A69-1602-64244',
       'A69-1602-64245', 'A69-1602-64246', 'A69-1602-64247',
       'A69-1602-64248', 'A69-1602-64249', 'A69-1602-64250',
       'A69-1602-64252', 'A69-1602-64255', 'A69-1602-64256',
       'A69-1602-64257', 'A69-1604-24858', 'A69-1604-27419',
       'A69-1604-27420', 'A69-1604-27421', 'A69-1604-27422',
       'A69-1604-27423', 'A69-1604-27424', 'A69-1604-27425',
       'A69-1604-27427', 'A69-1604-27430', 'A69-1604-27431',
       'A69-1604-27432', 'A69-1604-27433', 'A69-1604-27434',
       'A69-1604-27435', 'A69-1604-27436', 'A69-1604-27437',
       'A69-1604-27438', 'A69-1604-27439', 'A69-1604-27440',
       'A69-1604-27441', 'A69-1604-27442', 'A69-1604-27443',
       'A69-1604-27444', 'A69-1604-27445', 'A69-1604-27446',
       'A69-1604-27447', 'A69-1604-27448', 'A69-9001-16062',
       'A69-9001-20978',

In [5]:
## Filter each fish by date. 